source: [link](https://ai-bootcamp.ruangguru.com/learn/09_mlops/01_wandb/Text_Classification_on_GLUE.html)

In [ ]:
!pip install -q datasets transformers
!pip install transformers[torch]

In [ ]:
Requirement already satisfied: transformers[torch] in /usr/local/lib/python3.10/dist-packages (4.38.2)
Requirement already satisfied: filelock in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (3.13.1)
Requirement already satisfied: huggingface-hub<1.0,>=0.19.3 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (0.20.3)
Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (1.25.2)
Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (23.2)
Requirement already satisfied: pyyaml>=5.1 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (6.0.1)
Requirement already satisfied: regex!=2019.12.17 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (2023.12.25)
Requirement already satisfied: requests in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (2.31.0)
Requirement already satisfied: tokenizers<0.19,>=0.14 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (0.15.2)
Requirement already satisfied: safetensors>=0.4.1 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (0.4.2)
Requirement already satisfied: tqdm>=4.27 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (4.66.2)
Requirement already satisfied: torch in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (2.1.0+cu121)
Requirement already satisfied: accelerate>=0.21.0 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (0.27.2)
Requirement already satisfied: psutil in /usr/local/lib/python3.10/dist-packages (from accelerate>=0.21.0->transformers[torch]) (5.9.5)
Requirement already satisfied: fsspec>=2023.5.0 in /usr/local/lib/python3.10/dist-packages (from huggingface-hub<1.0,>=0.19.3->transformers[torch]) (2023.6.0)
Requirement already satisfied: typing-extensions>=3.7.4.3 in /usr/local/lib/python3.10/dist-packages (from huggingface-hub<1.0,>=0.19.3->transformers[torch]) (4.10.0)
Requirement already satisfied: sympy in /usr/local/lib/python3.10/dist-packages (from torch->transformers[torch]) (1.12)
Requirement already satisfied: networkx in /usr/local/lib/python3.10/dist-packages (from torch->transformers[torch]) (3.2.1)
Requirement already satisfied: jinja2 in /usr/local/lib/python3.10/dist-packages (from torch->transformers[torch]) (3.1.3)
Requirement already satisfied: triton==2.1.0 in /usr/local/lib/python3.10/dist-packages (from torch->transformers[torch]) (2.1.0)
Requirement already satisfied: charset-normalizer<4,>=2 in /usr/local/lib/python3.10/dist-packages (from requests->transformers[torch]) (3.3.2)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.10/dist-packages (from requests->transformers[torch]) (3.6)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.10/dist-packages (from requests->transformers[torch]) (2.0.7)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.10/dist-packages (from requests->transformers[torch]) (2024.2.2)
Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.10/dist-packages (from jinja2->torch->transformers[torch]) (2.1.5)
Requirement already satisfied: mpmath>=0.19 in /usr/local/lib/python3.10/dist-packages (from sympy->torch->transformers[torch]) (1.3.0)

In [ ]:
!pip -q install wandb
!wandb login
%env WANDB_PROJECT=indo-roberta-small-finetune-sentiment-analysis
%env WANDB_WATCH=true
%env WANDB_LOG_MODEL=true

In [ ]:
wandb: Currently logged in as: ayamerushia. Use `wandb login --relogin` to force relogin
env: WANDB_PROJECT=indo-roberta-small-finetune-sentiment-analysis
env: WANDB_WATCH=true
env: WANDB_LOG_MODEL=true

If you’re opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up here if you haven’t already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

In [ ]:
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

In [ ]:
4.38.2

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs here.

# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the 🤗 Transformers model to a text classification task of the GLUE Benchmark.

![Image](https://github.com/huggingface/notebooks/blob/master/examples/images/text_classification.png?raw=1)

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

We will see how to easily load the dataset for each one of those tasks and use the Trainer API to fine-tune a model on it. Each task is named by its acronym, with mnli-mm standing for the mismatched version of MNLI (so same training set as mnli but different validation and test sets):

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the Model Hub as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
task = "wnli"
model_checkpoint = "w11wo/indo-roberta-small"
batch_size = 64

# Loading the dataset

We will use the 🤗 Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions load_dataset and load_metric.

In [ ]:
from datasets import load_dataset, load_metric

Apart from mnli-mm being a special code, we can directly pass our task name to those functions. load_dataset will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
# actual_task = "mnli" if task == "mnli-mm" else task
# dataset = load_dataset("glue", actual_task)


dataset = load_dataset("indonlu", "smsa")
metric = load_metric('glue', "mnli")

In [ ]:
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for indonlu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/indonlu
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
<ipython-input-9-777f794771cd>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', "mnli")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(

The metric is an instance of datasets.Metric:

# Preprocessing the data

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

encoded_dataset = dataset.map(tokenize_function, batched=True)

# Fine-tuning the model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at w11wo/indo-roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-indonlu-smsa",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    overwrite_output_dir = True,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.005,
    warmup_steps = 1000,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
    report_to = 'wandb',
    run_name = 'roberta-mini-run-2'
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the Trainer to load the best model it saved (according to metric_name) at the end of training.

The last argument to setup everything so we can push the model to the Hub regularly during training. Remove it if you didn’t follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the hub_model_id argument to set the repo name (it needs to be the full name, including your namespace: for instance "sgugger/bert-finetuned-mrpc" or "huggingface/bert-finetuned-mrpc").

The last thing to define for our Trainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the Trainer:

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

You might wonder why we pass along the tokenizer when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model’s preferences regarding padding (to the left or right? with which token?). The tokenizer has a pad method that will do all of this right for us, and the Trainer will use it. You can customize this part by defining and passing your own data_collator which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the train method:

In [ ]:
 import numpy as np
trainer.train()

In [ ]:
wandb: Currently logged in as: ayamerushia. Use `wandb login --relogin` to force relogin

| Epoch | Training Loss | Validation Loss | Accuracy |
| --- | --- | --- | --- |
| 1 | No log | 0.650164 | 0.714286 |
| 2 | No log | 0.472029 | 0.812698 |
| 3 | 0.616800 | 0.451087 | 0.835714 |
| 4 | 0.616800 | 0.382544 | 0.853968 |
| 5 | 0.616800 | 0.365464 | 0.859524 |
| 6 | 0.295400 | 0.367193 | 0.868254 |
| 7 | 0.295400 | 0.383897 | 0.874603 |
| 8 | 0.295400 | 0.421972 | 0.870635 |
| 9 | 0.132800 | 0.449737 | 0.888095 |
| 10 | 0.132800 | 0.445541 | 0.886508 |

In [ ]:
Checkpoint destination directory indo-roberta-small-finetuned-indonlu-smsa/checkpoint-344 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory indo-roberta-small-finetuned-indonlu-smsa/checkpoint-688 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory indo-roberta-small-finetuned-indonlu-smsa/checkpoint-1032 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory indo-roberta-small-finetuned-indonlu-smsa/checkpoint-1376 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory indo-roberta-small-finetuned-indonlu-smsa/checkpoint-1720 already exists and is non-empty. Saving will proceed but saved results may be invalid.

In [ ]:
TrainOutput(global_step=1720, training_loss=0.31177856478580207, metrics={'train_runtime': 950.9864, 'train_samples_per_second': 115.669, 'train_steps_per_second': 1.809, 'total_flos': 2701973296566432.0, 'train_loss': 0.31177856478580207, 'epoch': 10.0})

We can check with the evaluate method that our Trainer did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

In [ ]:
{'eval_loss': 0.44973695278167725,
 'eval_accuracy': 0.888095238095238,
 'eval_runtime': 3.2955,
 'eval_samples_per_second': 382.345,
 'eval_steps_per_second': 6.069,
 'epoch': 10.0}

To see how your model fared you can compare it to the GLUE Benchmark leaderboard.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

In [ ]:
CommitInfo(commit_url='https://huggingface.co/ayameRushia/indo-roberta-small-finetuned-indonlu-smsa/commit/9c77581eec0a70db3a03de4600796e21ed57c7e9', commit_message='End of training', commit_description='', oid='9c77581eec0a70db3a03de4600796e21ed57c7e9', pr_url=None, pr_revision=None, pr_num=None)